In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%watermark -v -n -m -p numpy,scipy,sklearn,pandas

Sat Dec 22 2018 

CPython 3.6.6
IPython 6.5.0

numpy 1.15.3
scipy 1.1.0
sklearn 0.19.1
pandas 0.23.4

compiler   : GCC 4.8.2 20140120 (Red Hat 4.8.2-15)
system     : Linux
release    : 4.15.0-42-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 4
interpreter: 64bit


In [2]:
%matplotlib inline
import sys
import pandas as pd
import numpy as np
import seaborn as sns
import os
import nolds
import data
import mne

PROJ_ROOT = os.path.abspath(os.path.join(os.pardir))
DATA_ROOT = os.path.abspath(os.path.join(PROJ_ROOT, 'data'))
PROCESSED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'processed'))
RAW_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'raw'))
LABELED_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'labeled'))
DURATIONS_ROOT = os.path.abspath(os.path.join(DATA_ROOT, 'durations'))
sys.path.append(os.path.join(PROJ_ROOT, 'src'))
CHANNEL_NAMES = ['FP1', 'FP2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2',
                 'F7', 'F8', 'T3', 'T4', 'T5', 'T6', 'Fz', 'Cz', 'Pz']
META_COLUMN_NAMES = ['freq', 'RESP_4W', 'RESP_FIN', 'REMISE_FIN', 'AGE', 'SEX', 'M_1',
       'M_4', 'M_F', 'délka léčby', 'lék 1', 'lék 2', 'lék 3', 'lék 4']
META_FILE_NAME = 'DEP-POOL_Final_144.xlsx'
meta_df = pd.read_excel(os.path.join(RAW_ROOT, META_FILE_NAME), index_col='ID', names=META_COLUMN_NAMES)

raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '1a.fif'))
t = pd.DataFrame(raw_fif.get_data())
data = pd.DataFrame(np.transpose(t.values), columns=CHANNEL_NAMES)
data = np.transpose(data.values)
metapkl = pd.read_pickle(os.path.join(LABELED_ROOT, 'processed', 'meta', 'meta.pkl'))

Opening raw data file /home/miroslav/Source/thesis_project/data/processed/1a.fif...
This filename (/home/miroslav/Source/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
Isotrak not found
    Range : 0 ... 19104 =      0.000 ...    76.416 secs
Ready.


<ipython-input-2-6b478cf116b8>:25: RuntimeWarning: This filename (/home/miroslav/Source/thesis_project/data/processed/1a.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz or raw_tsss.fif.gz
  raw_fif = mne.io.read_raw_fif(os.path.join(PROCESSED_ROOT, '1a.fif'))


In [3]:
def prepare_dfs(col='lyap', kind='processed'):
    df = pd.read_pickle(os.path.join(LABELED_ROOT, kind, col, f'training_{col}.pkl')).dropna()
    df_bef = pd.read_pickle(os.path.join(LABELED_ROOT, kind, col, f'{col}_bef.pkl')).dropna()
    df_aft = pd.read_pickle(os.path.join(LABELED_ROOT, kind, col, f'{col}_aft.pkl')).dropna()
    return df, df_bef, df_aft
    
def prepare_resp_non(col='lyap'):
    df, df_bef, df_aft = prepare_dfs(col)
    return df[df['change'] >= df.quantile(0.66)['change']], df[df['change'] <= df.quantile(0.33)['change']]

col = 'all'
df, df_bef, df_aft = prepare_dfs(col)
# df_resp, df_non = prepare_resp_non(col)
df.loc[:, (slice(None), ('lyap', 'corr', 'dfa', 'sampen'))]

channel               FP1                                         FP2  \
feature              lyap        corr       dfa    sampen        lyap   
patient trial                                                           
1       a       0.0354587     1.06838  0.875958  0.591666  0.00136619   
        b       0.0399611    0.106497  0.800248   0.72875  0.00766953   
2       b       0.0371767    0.634254  0.437548  0.620112   0.0321007   
3       a       0.0386179     2.51507  0.341363  0.466363   0.0383885   
        b       0.0365023     2.48113  0.406266  0.518688   0.0370322   
4       a       0.0375807     2.87517  0.565936  0.597395   0.0384536   
        b       0.0363211     1.06902  0.648121  0.601317   0.0371695   
5       a       0.0351841    0.434594  0.812759  0.652824   0.0347834   
        b       0.0344521    0.459655   0.56224  0.632775   0.0347157   
6       b       0.0512405    0.293646  0.626166  0.704864   0.0501606   
7       a       0.0520428    0.321962  0.586863  0.649313   0.0491474   
        b        0.055538     1.25891  0.497486  0.612224   0.0543862   
8       b       0.0320772     2.46557  0.613183  0.489485   0.0348243   
9       a       0.0511946    0.303689  0.803456  0.672748   0.0503297   
        b        0.036028     1.00812  0.861972  0.584781   0.0526195   
10      a       0.0656164   0.0139393  0.688589  0.940555   0.0663356   
        b       0.0450889   0.0758747  0.756033  0.776302   0.0445457   
11      a       0.0274139    0.104895  0.579021  0.771227   0.0203739   
        b       0.0257492   0.0944609   0.77545  0.805837   0.0278177   
12      a      0.00846576     2.07888   1.09068  0.532951   0.0420494   
        b       0.0313727  0.00724362  0.682067  0.888382   0.0329091   
13      b       0.0592675   0.0360323  0.560523   0.76922    0.061401   
14      a       0.0341264    0.189369  0.842003  0.709296  0.00240275   
        b       0.0209462    0.618397  0.928748  0.611801   0.0254034   
15      a       0.0322275    0.228731  0.905694  0.709038   0.0222773   
        b       0.0206867    0.599345  0.893735  0.613149   0.0325639   
16      a       0.0330165    0.837145  0.465842  0.576177   0.0340929   
        b       0.0314038    0.612233  0.670771  0.618317   0.0331228   
17      a       0.0642885   0.0528201  0.454658  0.787714   0.0676036   
        b       0.0431283   0.0412609  0.756309  0.807832   0.0421583   
...                   ...         ...       ...       ...         ...   
117     a       0.0255894     0.19085  0.819095  0.734178   0.0421705   
        b       0.0440871   0.0922706  0.570014  0.778484   0.0384747   
118     a       0.0355514    0.017817  0.639167  0.875469   0.0398086   
119     a       0.0359397    0.473043  0.889105  0.667802   0.0351513   
        b       0.0281931    0.165703  0.820736  0.736928   0.0190477   
120     a       0.0270263    0.080314  0.414707  0.775412   0.0433082   
121     a       0.0202561    0.896134  0.940726  0.587278   0.0189725   
        b       0.0584668   0.0131312  0.536531   1.00172    0.024483   
122     a       0.0279221     0.21726   0.83934  0.680467   0.0258261   
        b       0.0267358    0.184354  0.549117  0.704016   0.0400417   
123     a       0.0360139    0.212727  0.720932  0.697911   0.0388183   
        b       0.0447839    0.153503   0.70432  0.719974   0.0334372   
124     a       0.0471553    0.249413   0.66091  0.698469   0.0484982   
        b       0.0422593   0.0739531  0.705418  0.753907   0.0460776   
125     a       0.0483008    0.105811  0.721033  0.742761   0.0500183   
126     a       0.0435048   0.0341872   0.68984  0.813593   0.0240085   
        b       0.0489387    0.180365  0.656238  0.712423    0.034579   
127     a       0.0311236    0.130674  0.670575   0.71524    0.036301   
        b       0.0430605   0.0351511   0.71863   0.78813   0.0465227   
128     a       0.0433045   0.0447943  0.766633  0.849587   0.0233451   
129     a       0.0488494    0.169712  0.773959  0.694048 

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import (SelectFromModel, RFE, SelectKBest, mutual_info_classif)
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm, grid_search, datasets

df, df_bef, df_aft = prepare_dfs('all')
X = df[CHANNEL_NAMES]
df = X.drop(metapkl.columns, axis=1, errors='ignore')
y = X.join(metapkl)['dep']

X_train, X_test, y_train, y_test = \
    train_test_split(
        X, y, test_size=0.2, random_state=213)

unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))
unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))
 
y_train = y_train.astype('int')

estimator = grid_search.GridSearchCV(svm.SVC(), {
    'C': [0.5, 1.0, 1.5, 2],
    'kernel':('linear', 'poly', 'rbf', 'sigmoid'),
    'shrinking': (True, False),
    'decision_function_shape' : ('ovo', 'ovr'),
})
selector = GeneticSelectionCV(estimator,
                                  cv=5,
                                  verbose=1,
                                  scoring="accuracy",
                                  n_population=50,
                                  crossover_proba=0.8,
                                  mutation_proba=0.2,
                                  n_generations=50,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  caching=True,
                                  n_jobs=-1)
selector.fit(X_train, y_train)
y_pred = selector.predict(X_test)
y_pred = y_pred.astype('int')
y_test = y_test.astype('int')

print("Accuracy score: %.2f" % accuracy_score(y_test, y_pred))
print('(tn, fp, fn, tp): ', confusion_matrix(y_test, y_pred).ravel())
print('Coefficients: \n', np.array(X.stack().columns)[selector.support_])

{-1: 66, 0: 63, 1: 63}
{-1: 18, 0: 15, 1: 15}


/home/miroslav/anaconda3/envs/thesis/lib/python3.6/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


Selecting features with genetic algorithm.
gen	nevals	avg                      	std                    	min                      	max                    
0  	50    	[ 0.31864685 47.38      ]	[0.01953794 4.66428987]	[ 0.27565142 39.        ]	[ 0.3663271 57.       ]


KeyboardInterrupt: 

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
KeyboardInterrupt
